## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Identify target variable and feature variables
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
target = 'IS_SUCCESSFUL'
features = application_df.drop(['EIN', 'NAME'], axis=1)

In [21]:
# Determine the number of unique values in each column.
unique_counts = features.nunique()

In [22]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = features['APPLICATION_TYPE'].value_counts()


In [23]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)


# Replace in dataframe
for app in application_types_to_replace:
    features['APPLICATION_TYPE'] = features['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
features['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = features['CLASSIFICATION'].value_counts()
replace_classification = list(classification_counts[classification_counts < 1000].index)

In [25]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = features['CLASSIFICATION'].value_counts()[features['CLASSIFICATION'].value_counts() > 1]

In [26]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 500
classifications_to_replace = list(classification_counts_gt1[classification_counts_gt1 < cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    features['CLASSIFICATION'] = features['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
features['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1458
C7000      777
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C6100        1
C2600        1
C1248        1
C1580        1
C1820        1
C1900        1
C2570        1
C1283        1
C3700        1
C2500        1
C1570        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encoded = pd.get_dummies(features)

In [28]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop(target, axis=1)
y = df_encoded[target]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_train)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
input_features = X_train.shape[1]

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                710       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, shuffle=True, verbose=2)

Epoch 1/50
858/858 - 0s - loss: 0.5435 - accuracy: 0.7346 - 328ms/epoch - 382us/step
Epoch 2/50
858/858 - 0s - loss: 0.5436 - accuracy: 0.7344 - 319ms/epoch - 372us/step
Epoch 3/50
858/858 - 0s - loss: 0.5436 - accuracy: 0.7350 - 318ms/epoch - 371us/step
Epoch 4/50
858/858 - 0s - loss: 0.5435 - accuracy: 0.7349 - 317ms/epoch - 370us/step
Epoch 5/50
858/858 - 0s - loss: 0.5435 - accuracy: 0.7343 - 318ms/epoch - 371us/step
Epoch 6/50
858/858 - 0s - loss: 0.5436 - accuracy: 0.7349 - 319ms/epoch - 372us/step
Epoch 7/50
858/858 - 0s - loss: 0.5435 - accuracy: 0.7362 - 317ms/epoch - 370us/step
Epoch 8/50
858/858 - 0s - loss: 0.5432 - accuracy: 0.7352 - 320ms/epoch - 372us/step
Epoch 9/50
858/858 - 0s - loss: 0.5434 - accuracy: 0.7348 - 326ms/epoch - 380us/step
Epoch 10/50
858/858 - 0s - loss: 0.5431 - accuracy: 0.7346 - 313ms/epoch - 365us/step
Epoch 11/50
858/858 - 0s - loss: 0.5434 - accuracy: 0.7348 - 306ms/epoch - 357us/step
Epoch 12/50
858/858 - 0s - loss: 0.5435 - accuracy: 0.7352 - 30

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5568 - accuracy: 0.7267 - 141ms/epoch - 658us/step
Loss: 0.5567781925201416, Accuracy: 0.7266764044761658


In [37]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")